In [13]:
import math
# ec_id = 511510650
eid = 501836392 #501886692 three_session_B 501836392 three_session_A 501717543 three_session_C
c1 = 517419703 
c2 = 517419533 
use_events = True

In [14]:
import os
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'
if not os.path.isdir(drive_path):
    drive_path = '/media/alihamodi/Brain2019/allen-brain-observatory/visual-coding-2p/'
    
import warnings;
warnings.simplefilter('ignore');
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from utils.data_wrappers import *
from utils.polardata import *
from utils.polarplot import *
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [15]:

"""For one experiment, get the movie presentations, for each one get temporal correlation of
the two cells, then average all these temporal correlations.
Note about spontaneous:
I checked that sesh A and B just have 1 spontaenous preso, session C have 2.
So, only the big grey chunks are counted. The small ones aren't.
Note about movies:
Each row in stim table for the movie is a frame of a particular trial
Trial number = repeat. 
I'm just using natural movie 1 because it exists in all experiments.
See http://alleninstitute.github.io/AllenSDK/_static/container_session_layout.png
"""
data_set = boc.get_ophys_experiment_data(eid)

try: 
    cidxs = data_set.get_cell_specimen_indices([c1, c2])
except Exception as inst:
    print(inst)

if use_events:
  events = boc.get_ophys_experiment_events(ophys_experiment_id=eid)
  cidx1 = cidxs[0]
  cidx2 = cidxs[1]
  events1 = events[cidx1,:]
  events2 = events[cidx2,:]
else: 
  timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[c1])
  events1 = dff[0,:]
  timestamps, dff = data_set.get_dff_traces(cell_specimen_ids=[c2])
  events2 = dff[0,:]
"""
'natural_movie_one',
'natural_movie_three',
'natural_movie_two',
'spontaneous'
"""
stim_table = data_set.get_stimulus_table('natural_movie_one') 
c1_trial_results = []
c2_trial_results = []
for trial_i in range(stim_table.repeat.max() + 1):
    start = stim_table[stim_table.repeat==trial_i].start.min()
    end = stim_table[stim_table.repeat==trial_i].start.max()
    c1_trial_result = events1[start:end].mean()
    c2_trial_result = events2[start:end].mean()
    c1_trial_results.append(c1_trial_result)
    c2_trial_results.append(c2_trial_result)
print(c1_trial_results, c2_trial_results)
corr, p_value = pearsonr(c1_trial_results, c2_trial_results)
corr

[0.0001603135947213615, 0.00013505601776856786, 6.163491096731715e-05, 6.579635567553524e-05, 0.0003792031192549915, 0.0001463015617968356, 5.202832565913244e-05, 0.0002693655542786989, 0.0, 0.00012311738651705486] [0.000684910201388231, 0.0004941963345247459, 0.002225980057041331, 0.0013739206231091782, 0.0009550773157904573, 0.0011127031385570131, 9.169894116931984e-05, 0.0010059844543862243, 0.00043158589932003474, 0.0005727159410732]


0.04776686696429075